# 第三章 储存

 - [一、对话缓存储存](#二、对话缓存储存)
     - [1.1 初始化对话模型](#1.1-初始化对话模型)
     - [1.2 对话](#1.2-对话)
     - [1.3 查看储存缓存](#1.3-查看储存缓存)
     - [1.4 直接添加内容到储存缓存](#1.4-直接添加内容到储存缓存)
     - [1.5 总结](#1.5-总结)
 - [二、对话缓存窗口储存](#二、对话缓存窗口储存)
     - [2.1 添加两轮对话到窗口储存](#2.1-添加两轮对话到窗口储存)
     - [32.2 在对话链中应用窗口储存](#2.2-在对话链中应用窗口储存)
 - [三、对话token缓存储存](#三、对话token缓存储存)
 - [四、对话摘要缓存储存](#四、对话摘要缓存储存)
     - [4.1 使用对话摘要缓存储存](#4.1-使用对话摘要缓存储存)
     - [4.2 基于对话摘要缓存储存的对话链](#4.2-基于对话摘要缓存储存的对话链)


当你与那些语言模型进行交互的时候，他们不会记得你之前和他进行的交流内容，这在我们构建一些应用程序（如聊天机器人）的时候，是一个很大的问题 -- 显得不够智能！因此，在本节中我们将介绍 LangChain 中的储存模块，即如何将先前的对话嵌入到语言模型中的，使其具有连续对话的能力。

当使用 LangChain 中的储存(Memory)模块时，它可以帮助保存和管理历史聊天消息，以及构建关于特定实体的知识。这些组件可以跨多轮对话储存信息，并允许在对话期间跟踪特定信息和上下文。

LangChain 提供了多种储存类型。其中，缓冲区储存允许保留最近的聊天消息，摘要储存则提供了对整个对话的摘要。实体储存 则允许在多轮对话中保留有关特定实体的信息。这些记忆组件都是模块化的，可与其他组件组合使用，从而增强机器人的对话管理能力。储存模块可以通过简单的API调用来访问和更新，允许开发人员更轻松地实现对话历史记录的管理和维护。

此次课程主要介绍其中四种储存模块，其他模块可查看文档学习。
- 对话缓存储存 (ConversationBufferMemory）
- 对话缓存窗口储存 (ConversationBufferWindowMemory）
- 对话令牌缓存储存 (ConversationTokenBufferMemory）
- 对话摘要缓存储存 (ConversationSummaryBufferMemory）

在LangChain中，储存 指的是大语言模型（LLM）的短期记忆。为什么是短期记忆？那是因为LLM训练好之后 (获得了一些长期记忆)，它的参数便不会因为用户的输入而发生改变。当用户与训练好的LLM进行对话时，LLM会暂时记住用户的输入和它已经生成的输出，以便预测之后的输出，而模型输出完毕后，它便会“遗忘”之前用户的输入和它的输出。因此，之前的这些信息只能称作为LLM的短期记忆。  
  
为了延长LLM短期记忆的保留时间，则需要借助一些外部储存方式来进行记忆，以便在用户与LLM对话中，LLM能够尽可能的知道用户与它所进行的历史对话信息。

## 一、对话缓存储存


### 1.1 初始化对话模型

In [1]:
import os
from langchain_openai import ChatOpenAI
from langchain.chains.conversation.base import ConversationChain
from langchain.memory.buffer import ConversationBufferMemory

# 这里我们将参数temperature设置为0.0，从而减少生成答案的随机性。
# 如果你想要每次得到不一样的有新意的答案，可以尝试调整该参数。

API_KEY = os.environ.get("CHAT_ANYWHERE_API_KEY")
BASE_URL = "https://api.chatanywhere.com.cn/v1"

In [2]:
llm = ChatOpenAI(temperature=0.0, base_url=BASE_URL, api_key=API_KEY)
memory = ConversationBufferMemory()
conversation = ConversationChain(llm=llm, memory=memory, verbose=True)

conversation

ConversationChain(verbose=True, llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x10a040a90>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x10a011b40>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_api_base='https://api.chatanywhere.com.cn/v1', openai_proxy=''))

### 1.2 对话

In [3]:
conversation.predict(input="你好, 我叫皮皮鲁")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 你好, 我叫皮皮鲁
AI:

> Finished chain.


' 你好皮皮鲁！我是一个AI助手，很高兴认识你。你有什么问题想问我吗？'

In [4]:
conversation.predict(input="1+1等于多少？")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 你好, 我叫皮皮鲁
AI:  你好皮皮鲁！我是一个AI助手，很高兴认识你。你有什么问题想问我吗？
Human: 1+1等于多少？
AI:

> Finished chain.


'1加1等于2。这是一个基本的数学问题，答案是2。你还有其他问题吗？'

In [5]:
conversation.predict(input="我叫什么名字？")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 你好, 我叫皮皮鲁
AI:  你好皮皮鲁！我是一个AI助手，很高兴认识你。你有什么问题想问我吗？
Human: 1+1等于多少？
AI: 1加1等于2。这是一个基本的数学问题，答案是2。你还有其他问题吗？
Human: 我叫什么名字？
AI:

> Finished chain.


'你叫皮皮鲁。你之前告诉我你的名字是皮皮鲁。有什么其他问题想问我吗？'

### 1.3 查看储存缓存

In [6]:
print(memory.buffer)

Human: 你好, 我叫皮皮鲁
AI:  你好皮皮鲁！我是一个AI助手，很高兴认识你。你有什么问题想问我吗？
Human: 1+1等于多少？
AI: 1加1等于2。这是一个基本的数学问题，答案是2。你还有其他问题吗？
Human: 我叫什么名字？
AI: 你叫皮皮鲁。你之前告诉我你的名字是皮皮鲁。有什么其他问题想问我吗？


In [7]:
print(memory.load_memory_variables({}))

{'history': 'Human: 你好, 我叫皮皮鲁\nAI:  你好皮皮鲁！我是一个AI助手，很高兴认识你。你有什么问题想问我吗？\nHuman: 1+1等于多少？\nAI: 1加1等于2。这是一个基本的数学问题，答案是2。你还有其他问题吗？\nHuman: 我叫什么名字？\nAI: 你叫皮皮鲁。你之前告诉我你的名字是皮皮鲁。有什么其他问题想问我吗？'}


### 2.3 直接添加内容到储存缓存

In [8]:
memory = ConversationBufferMemory()
memory.save_context({"input": "你好，我叫皮皮鲁"}, {"output": "你好啊，我叫鲁西西"})
memory.load_memory_variables({})

{'history': 'Human: 你好，我叫皮皮鲁\nAI: 你好啊，我叫鲁西西'}

In [9]:
memory.save_context({"input": "Not much, just hanging"}, {"output": "Cool"})

In [10]:
memory.load_memory_variables({})

{'history': 'Human: 你好，我叫皮皮鲁\nAI: 你好啊，我叫鲁西西\nHuman: Not much, just hanging\nAI: Cool'}

In [11]:
memory.save_context({"input": "很高兴和你成为朋友！"}, {"output": "是的，让我们一起去冒险吧！"})
memory.load_memory_variables({})

{'history': 'Human: 你好，我叫皮皮鲁\nAI: 你好啊，我叫鲁西西\nHuman: Not much, just hanging\nAI: Cool\nHuman: 很高兴和你成为朋友！\nAI: 是的，让我们一起去冒险吧！'}

### 2.3 总结

当我们在使用大型语言模型进行聊天对话时，**大型语言模型本身实际上是无状态的。语言模型本身并不记得到目前为止的历史对话**。每次调用API结点都是独立的。储存(Memory)可以储存到目前为止的所有术语或对话，并将其输入或附加上下文到LLM中用于生成输出。如此看起来就好像它在进行下一轮对话的时候，记得之前说过什么。


## 二、对话缓存窗口储存
  
随着对话变得越来越长，所需的内存量也变得非常长。将大量的tokens发送到LLM的成本，也会变得更加昂贵,这也就是为什么API的调用费用，通常是基于它需要处理的tokens数量而收费的。
  
针对以上问题，LangChain也提供了几种方便的储存方式来保存历史对话。其中，对话缓存窗口储存只保留一个窗口大小的对话。它只使用最近的n次交互。这可以用于保持最近交互的滑动窗口，以便缓冲区不会过大

### 2.1 添加两轮对话到窗口储存

In [16]:
from langchain.memory import ConversationBufferWindowMemory

# k=1表明只保留一个对话记忆
memory = ConversationBufferWindowMemory(k=1)
memory.save_context({"input": "你好，我叫皮皮鲁"}, {"output": "你好啊，我叫鲁西西"})
memory.save_context({"input": "很高兴和你成为朋友！"}, {"output": "是的，让我们一起去冒险吧！"})
memory.load_memory_variables({})

{'history': 'Human: 很高兴和你成为朋友！\nAI: 是的，让我们一起去冒险吧！'}

### 2.2 在对话链中应用窗口储存

注意此处！由于这里用的是一个窗口的记忆，因此只能保存一轮的历史消息，因此AI并不能知道你第一轮对话中提到的名字，他最多只能记住上一轮（第二轮）的对话信息

In [17]:
llm = ChatOpenAI(temperature=0.0, base_url=BASE_URL, api_key=API_KEY)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(llm=llm, memory=memory, verbose=False)
print(conversation.predict(input="你好, 我叫皮皮鲁"))
print(conversation.predict(input="1+1等于多少？"))
conversation.predict(input="我叫什么名字？")

 你好皮皮鲁！我是一个AI助手，很高兴认识你。你有什么问题想问我吗？
1加1等于2。这是一个基本的数学问题，答案是2。你还有其他问题吗？


'抱歉，我不知道你的名字。我只是一个人工智能程序，无法知道你的个人信息。有其他问题我可以帮你解答吗？'

## 三、对话token缓存储存

使用对话token缓存记忆，内存将限制保存的token数量。如果token数量超出指定数目，它会切掉这个对话的早期部分
以保留与最近的交流相对应的token数量，但不超过token限制。

In [76]:
!pip install -q tiktoken            

In [18]:
from langchain.memory import ConversationTokenBufferMemory

# 指定llm类型的token计算方式
llm = ChatOpenAI(temperature=0.0, base_url=BASE_URL, api_key=API_KEY)
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=30)
memory.save_context({"input": "朝辞白帝彩云间，"}, {"output": "千里江陵一日还。"})
memory.save_context({"input": "两岸猿声啼不住，"}, {"output": "轻舟已过万重山。"})

# 前面超出的的token已经被舍弃了！！！
memory.load_memory_variables({})

{'history': 'AI: 轻舟已过万重山。'}

补充

ChatGPT使用一种基于字节对编码（Byte Pair Encoding，BPE）的方法来进行tokenization（将输入文本拆分为token）。BPE是一种常见的tokenization技术，它将输入文本分割成较小的子词单元。 

OpenAI在其官方GitHub上公开了一个最新的开源Python库 [tiktoken](https://github.com/openai/tiktoken)，这个库主要是用来计算tokens数量的。相比较HuggingFace的tokenizer，其速度提升了好几倍。
在线的token计算方式：[OpenAI Platform Tokenizer](https://platform.openai.com/tokenizer)

具体token计算方式,特别是汉字和英文单词的token区别，具体课参考[知乎文章](https://www.zhihu.com/question/594159910) 。

## 四、对话摘要缓存储存

对话摘要缓存储存，**使用LLM编写到目前为止历史对话的摘要**，并将其保存

### 4.1 使用对话摘要缓存储存

创建一个长字符串，其中包含某人的日程安排

In [19]:
from langchain.chains import ConversationChain
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory


# 创建一个长字符串
schedule = "在八点你和你的产品团队有一个会议。 \
你需要做一个PPT。 \
上午9点到12点你需要忙于LangChain。\
Langchain是一个有用的工具，因此你的项目进展的非常快。\
中午，在意大利餐厅与一位开车来的顾客共进午餐 \
走了一个多小时的路程与你见面，只为了解最新的 AI。 \
确保你带了笔记本电脑可以展示最新的 LLM 样例."

llm = ChatOpenAI(temperature=0.0, base_url=BASE_URL, api_key=API_KEY)
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)  # 调用API生成摘要
memory.save_context({"input": "你好，我叫皮皮鲁"}, {"output": "你好啊，我叫鲁西西"})
memory.save_context({"input": "很高兴和你成为朋友！"}, {"output": "是的，让我们一起去冒险吧！"})
memory.save_context({"input": "今天的日程安排是什么？"}, {"output": f"{schedule}"})

memory.load_memory_variables({})

{'history': 'System: The human and AI introduce themselves in Chinese and become friends. They plan their day, including a meeting with the product team, working on LangChain, and having lunch with a customer interested in AI. The AI emphasizes the importance of being prepared to showcase the latest LLM examples.'}

### 4.2 基于对话摘要缓存储存的对话链
基于上面的对话摘要缓存储存，新建一个对话链

In [20]:
conversation = ConversationChain(llm=llm, memory=memory, verbose=True)
conversation.predict(input="展示什么样的样例最好呢？使用中文回答")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI introduce themselves in Chinese and become friends. They plan their day, including a meeting with the product team, working on LangChain, and having lunch with a customer interested in AI. The AI emphasizes the importance of being prepared to showcase the latest LLM examples.
Human: 展示什么样的样例最好呢？使用中文回答
AI:

> Finished chain.


'最好展示一些涉及自然语言处理和语言生成的例子，比如文本摘要、对话生成和情感分析等。这些例子能够展示LangChain在处理自然语言任务方面的强大能力。'

In [21]:
memory.load_memory_variables({})  # 摘要记录更新了

{'history': 'System: The human and AI introduce themselves in Chinese and become friends. They plan their day, including a meeting with the product team, working on LangChain, and having lunch with a customer interested in AI. The AI emphasizes the importance of being prepared to showcase the latest LLM examples.\nHuman: 展示什么样的样例最好呢？使用中文回答\nAI: 最好展示一些涉及自然语言处理和语言生成的例子，比如文本摘要、对话生成和情感分析等。这些例子能够展示LangChain在处理自然语言任务方面的强大能力。'}